In [3]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import torch.optim as optim

Dataset

In [4]:
TABULAR_COLS = ["bedrooms","bathrooms","sqft_living","floors","grade"]
TARGET_COL = "log_price"

class HousePriceDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = f"{self.img_dir}/{row['id']}.jpg"
        if not os.path.exists(img_path):
            return None

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        tabular = torch.tensor(row[TABULAR_COLS].values, dtype=torch.float32)
        price = torch.tensor(row[TARGET_COL], dtype=torch.float32)

        return image, tabular, price


Model

In [5]:
class MultimodalPriceModel(nn.Module):
    def __init__(self, num_tabular):
        super().__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()

        self.tabular_fc = nn.Sequential(
            nn.Linear(num_tabular, 32),
            nn.ReLU()
        )

        self.regressor = nn.Sequential(
            nn.Linear(512 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, img, tab):
        img_feat = self.cnn(img)
        tab_feat = self.tabular_fc(tab)
        return self.regressor(torch.cat([img_feat, tab_feat], 1)).squeeze()


Training

In [ ]:
df = pd.read_csv("data/clean_train.csv")
image_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

dataset = HousePriceDataset(df, "data/images/train", image_transform)
dataset_small = Subset(dataset, range(14000))

loader = DataLoader(dataset_small, batch_size=4, shuffle=True)

model = MultimodalPriceModel(len(TABULAR_COLS))
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(2):
    total = 0
    for img, tab, y in loader:
        optimizer.zero_grad()
        pred = model(img, tab)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total += loss.item()
    print(f"Epoch {epoch+1}, Loss {total/len(loader):.4f}")


In [ ]:
from sklearn.metrics import r2_score

df_train=pd.read_csv('/Users/prashantmaurya/Desktop/Satellite_Property_Valuation/train_actual_vs_predicted.csv')

y_true = df_train["actual_price"].values
y_pred = df_train["predicted_price"].values

r2 = r2_score(y_true, y_pred)
print("R2 score:", r2)


In [6]:
from sklearn.metrics import mean_squared_error, r2_score

df_eval = pd.read_csv("/Users/prashantmaurya/Desktop/Satellite_Property_Valuation/train_actual_vs_predicted.csv")

y_true = df_eval["actual_price"].values
y_pred = df_eval["predicted_price"].values

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print("Evaluation Metrics (Train Dataset)")
print("----------------------------------")
print(f"RMSE : {rmse:,.2f}")
print(f"R² Score : {r2:.4f}")


Evaluation Metrics (Train Dataset)
----------------------------------
RMSE : 139,578.85
R² Score : 0.7549


In [8]:
!pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 39.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 42.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.


In [9]:
import cv2
import matplotlib.pyplot as plt

In [20]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def generate(self, image, tabular):
        self.model.zero_grad()
        output = self.model(image, tabular)
        output.backward()

        grads = self.gradients.mean(dim=[2, 3], keepdim=True)
        cam = (grads * self.activations).sum(dim=1)
        cam = torch.relu(cam)

        cam = cam.squeeze().detach().cpu().numpy()
        cam = cv2.resize(cam, (224, 224))
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

        return cam


In [ ]:
# Pick one sample
idx = 10
img, tab, _ = dataset[idx]

img = img.unsqueeze(0).to(device)
tab = tab.unsqueeze(0).to(device)

# ResNet last conv layer
target_layer = model.cnn.layer4[-1].conv2

gradcam = GradCAM(model, target_layer)
heatmap = gradcam.generate(img, tab)

# Convert image tensor to numpy
img_np = img.squeeze().permute(1, 2, 0).cpu().numpy()
img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())

# Overlay
plt.figure(figsize=(5,5))
plt.imshow(img_np)
plt.imshow(heatmap, cmap='jet', alpha=0.5)
plt.axis("off")
plt.title("Grad-CAM: Model Attention on Satellite Image")
plt.show()
